# 3. Wrapping a Smolagents Agent into an ACP Server

We will now create another ACP agent built with Smolagents. This second agent will be able to search the web to handle travel related questions for customers.  We will wrap the agent definition in an ACP server similarly to how we did with the first RAG agent.

## 3.1. Wrap the Agent in ACP Server

To define our agent, we will use [CodeAgent](https://smolagents.org/docs/agents-guided-tour/) from the Smolagents library. This type of agent writes and executes Python code at every step. For this agent, we will use two tools provided by Smolagents:

- DuckDuckGoSearchTool: performs a web search using DuckDuckGo browser
- VisitWebpageTool: can visit and extract content from web pages

The agent is wrapped in an ACP server using `@server.agent()` decorator. The server will also run locally using a different port number: 8001. Run the following cell to save the agent as `smolagents_server.py` under `travel_acp_project`.

In [1]:
%%writefile ../travel_acp_project/src/smolagents_server.py
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import Context, RunYield, RunYieldResume, Server
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool
import logging
from dotenv import load_dotenv

load_dotenv()

server = Server()

# Configure the LLM
model = LiteLLMModel(
    model_id="openai/gpt-4",
    max_tokens=2048
)

@server.agent()
async def travel_research_agent(input: list[Message], context: Context) -> AsyncGenerator[RunYield, RunYieldResume]:
    """
    This is a CodeAgent that supports travelers with up-to-date information about travel destinations,
    visa requirements, best seasons to visit, and real-time tourism insights. It uses web search and
    web page browsing tools to answer travel-related questions.
    """
    agent = CodeAgent(
        tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
        model=model
    )

    prompt = input[0].parts[0].content
    response = agent.run(prompt)

    yield Message(parts=[MessagePart(content=str(response))])

if __name__ == "__main__":
    server.run(port=8000)

Overwriting ../my_acp_project/smolagents_server.py


## 3.2. Run the Travel Planner ACP Server

Now to activate our configured ACP agent, we also need to run our agent from another terminal.

Open the second terminal by running the following commands:
- `cd travel_acp_project`: navigate to directory where we created `travel_acp_project`.
- `uv venv`: to create a virtual environment
- `uv sync`: to install the dependencies defined in `pyproject.toml` file.
- `.venv\Scripts\activate`: to activate the virtual environment.
- Type `uv run python src/smolagents_server.py` to run the server and activate our ACP agent.

## 3.3. Adding MCP to the Travel Planner Server

We will add another agent to the Travel Planner Server that uses MCP to get access to tools:
- We will first define an MCP server using `FastMCP` that runs using the `stdio` transport (i.e., it runs locally) and exposes one tool;
- We will then update the travel planner server to include a second agent;
- We will define the agent using the ToolCallingAgent of Smolagents and pass to it the tool exposed by the MCP server;
- We will finally activate the agent and interact with it using an ACP client.

Run this cell to create the file `mcpserver.py` inside the folder `travel_acp_project`. The dependencies in the `travel_acp_project` have been also updated so we can run this MCP server using `uv run`.

In [ ]:
%%writefile ../travel_acp_project/src/mcpserver.py
from colorama import Fore
from mcp.server.fastmcp import FastMCP
import json
import requests

mcp = FastMCP("travelagencyserver")

@mcp.tool()
def list_travel_agencies(city: str) -> str:
    """
    This tool returns a list of travel agencies located in the given city.

    Args:
        city (str): The city name to search agencies for. Example: "Hyderabad"

    Returns:
        str: A list of travel agencies matching the city.
        Example: '[{"name": "GlobeTrotters Hyderabad", "rating": 4.6, "contact": "9876543210"}, ...]'
    """
    url = 'https://raw.githubusercontent.com/vinayak-me/travel-agency-mock-data/main/agencies.json'
    resp = requests.get(url)
    agencies = json.loads(resp.text)

    matched = [a for a in agencies if a['city'].lower() == city.lower()]
    return json.dumps(matched, indent=2)

if __name__ == "__main__":
    mcp.run(transport="stdio")
